# DecisionTree Assignment - 20기 황태연

물음표 친 부분을 채우고 코드에 대한 주석을 자세하게 달아주세요!

- 제출자: 20기 황태연
- 제출 일자: 2023.08.08. (화)

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

$$
Gini(D_i) = 1-\sum_j P_{j}^2
$$

In [42]:
def get_gini(df, label):
    # label에 들어있는 분류의 종류를 저장한다.
    classes = df[label].unique() # ['yes', 'no']
    
    # 데이터의 개수를 구한다.
    N = len(df)
    
    gini = 1
    # (P_j)**2를 gini = 1 에서 빼준다.
    for class_ in classes:
        gini -= (sum(df[label]==class_) / N)**2
        
    return gini

In [43]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [44]:
from itertools import combinations

def get_binary_split(df, attribute):
    
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 
    
    result = []
    for i in range(1, len(uniques)): # 1부터 len(uniques)-1까지
        for combination in combinations(uniques, i): # i개의 원소의 부분집합을 result에 추가
            result.append(list(combination))
    
    return result

In [45]:
get_binary_split(pd_data,'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

- Gini index를 구하는 식은 다음과 같다.
$$
Gini(D) = \sum_i \frac{|D_i|}{|D|}Gini(D_i)
$$

In [47]:
def get_attribute_gini_index(df, attribute, label):
    
    result = {}
    binary_split = get_binary_split(df, attribute)
    n = len(df)
    
    for key in binary_split:
        # df[attribute] 중 key 안에 있는 원소만 추출
        key_df = df[df[attribute].isin(key)]
        # df[attribute] 중 key 안에 없는 원소만 추출
        not_key_df = df[~df[attribute].isin(key)]
        
        D1 = len(key_df) # df[attribute] 중 key 안에 있는 원소의 개수
        D2 = n-D1 # df[attribute] 중 key 안에 없는 원소의 개수
        
        gini = 0
        
        gini_D1 = get_gini(key_df, label) # Gini(D_1)
        gini_D2 = get_gini(not_key_df, label) # Gini(D_2)
        
        gini = (D1/n)*gini_D1 + (D2/n)*gini_D2
        
        result[tuple(key)] =  gini
    return result

In [48]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{('high',): 0.4428571428571429,
 ('medium',): 0.4583333333333333,
 ('low',): 0.45,
 ('high', 'medium'): 0.45,
 ('high', 'low'): 0.4583333333333333,
 ('medium', 'low'): 0.4428571428571429}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [49]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

(('high',), 0.4428571428571429)

In [50]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

('high',)

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [58]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    gini = min(get_attribute_gini_index(pd_data, feature, 'class_buys_computer').items())[1]
    print(f'Minimum Gini Index of {feature} : {gini:.4f}')

Minimum Gini Index of age : 0.3571
Minimum Gini Index of income : 0.4429
Minimum Gini Index of student : 0.3673
Minimum Gini Index of credit_rating : 0.4286


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [59]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{('youth',): 0.3936507936507937,
 ('middle_aged',): 0.35714285714285715,
 ('senior',): 0.45714285714285713,
 ('youth', 'middle_aged'): 0.45714285714285713,
 ('youth', 'senior'): 0.35714285714285715,
 ('middle_aged', 'senior'): 0.3936507936507937}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [72]:
from math import log2

def getEntropy(df, feature) :
    """
    수식을 참고하여,
    데이터프레임 df에서 특정 feature에 대해 
    엔트로피를 구하는 함수를 작성해주세요.
    """
    # feature의 종류를 구합니다.
    features = df[feature].unique()
    
    # df의 데이터 개수를 구합니다.
    N = len(df)
    
    p = []
    # p_i를 구합니다.
    for f in features:
        n = sum(df[feature] == f)
        p.append(n/N)
    
    # Entropy를 계산합니다.
    p = np.array(p)
    H = -sum(p*np.log2(p))
    return H

In [73]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

- 각 변수의 엔트로피는 다음과 같이 구한다.
$$
E = \sum_i \frac{|S_i|}{|S|}H(S_I)
$$

- Information Gain은 다음과 같이 구한다.
$$
Gain = H(S) - E
$$

In [89]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :
        
    result = {}
    
    # 데이터의 개수를 구한다.
    N = len(df)
    
    # feature를 제외한 다른 변수들을 저장한다.
    other_features = df.drop(columns=[feature]).columns
    
    for f in other_features:
        # f 변수의 종류를 모두 구한다.
        s = df[f].unique()
        
        # 목표 변수의 Entropy를 구한 뒤 다른 변수의 Entropy를 빼준다.
        Entropy = getEntropy(df, feature)
        for s_i in s:
            # s_i에 해당하는 데이터만 담은 df를 생성한다.
            filtered_df = df[df[f] == s_i]
            
            # |s_i|를 구한다.
            n = len(filtered_df)
            
            # H(s_i)를 Entropy에 더한다.
            Entropy -= (n/N)*getEntropy(filtered_df, feature)
            
        result[f] = Entropy
    
    return result

In [90]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234164,
 'credit_rating': 0.048127030408269544}